In [1]:
import torch

torch.load("/home/hrc/gsoc/batch_1_0/data21000_s14.pt")

Data(x=[316, 6], edge_index=[2, 6634], edge_attr=[6634, 4], y=[6634])

In [2]:
# from torch_geometric.data import Data
# from torch_geometric.loader import DataLoader

# data_list = [Data(...), ..., Data(...)]
# loader = DataLoader(data_list, batch_size=32)

In [3]:
from pathlib import Path

from torch.utils.data import Dataset  # not the one from PyG!
from torch_geometric.loader import DataLoader
import torch.nn.functional as F
from torch_geometric.nn import GCNConv


import torch


class MyDataset(Dataset):
    def __init__(self, path: Path):
        super().__init__()
        self.graphs = list(path.glob("*.pt"))
    
    def __getitem__(self, idx):
        return torch.load(self.graphs[idx])
    
    def __len__(self) -> int:
        return len(self.graphs)


dataset = MyDataset(Path("/home/hrc/gsoc/batch_1_0/"))
loader = DataLoader(dataset, batch_size=32)

In [13]:
data = torch.load("/home/hrc/gsoc/batch_1_0/data21000_s14.pt")
data

Data(x=[316, 6], edge_index=[2, 6634], edge_attr=[6634, 4], y=[6634])

In [5]:
print(f'x = {data.x.shape}')
print(data.x)

x = torch.Size([316, 6])
tensor([[  0.0684,  -0.9035,  -1.4980,  -3.7805, -13.9569,  -4.3680],
        [  0.0686,  -0.9035,  -1.5020,  -3.7805, -13.9203,  -4.3556],
        [  0.1340,  -0.8955,  -1.4980,  -3.1095,  -7.0661,  -2.4080],
        ...,
        [  0.1050,  -0.9137,   1.5020,   3.3548,  -9.1744,  -2.5514],
        [  0.1128,  -0.8964,   1.4980,   3.2808,  -8.3998,  -2.8340],
        [  0.1047,  -0.9137,   1.4980,   3.3548,  -9.1993,  -2.5569]])


In [6]:
print(f'edge_index = {data.edge_index.shape}')
print(data.edge_index)

edge_index = torch.Size([2, 6634])
tensor([[  0,   0,   1,  ..., 313, 315, 314],
        [  3,   4,   3,  ..., 311, 311, 312]])


In [7]:
print(f'edge_attributes = {data.edge_attr.shape}')
print(data.edge_attr)

edge_attributes = torch.Size([6634, 4])
tensor([[-0.0092, -0.0007,  0.2000,  0.0025],
        [-0.0090, -0.0007,  0.1960,  0.0022],
        [-0.0094, -0.0007,  0.2040,  0.0024],
        ...,
        [-0.0140,  0.0012, -0.2040,  0.0045],
        [-0.0138,  0.0012, -0.2000,  0.0046],
        [-0.0149, -0.0011, -0.1960,  0.0038]])


In [14]:
print(f'y = {data.y.shape}')
print(data.y)

y = torch.Size([6634])
tensor([1., 1., 1.,  ..., 1., 1., 1.])


In [15]:
from torch_geometric.utils import to_dense_adj

A = to_dense_adj(data.edge_index)[0].numpy().astype(int)
print(f'A = {A.shape}')
print(A)

A = (316, 316)
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [16]:
print(f'y = {data.y.shape}')
print(data.y)

y = torch.Size([6634])
tensor([1., 1., 1.,  ..., 1., 1., 1.])


In [17]:
print(f'Edges are directed: {data.is_directed()}')
print(f'Graph has isolated nodes: {data.has_isolated_nodes()}')
print(f'Graph has loops: {data.has_self_loops()}')


Edges are directed: True
Graph has isolated nodes: False
Graph has loops: False


In [18]:
# Finding edge embeddings

# x = ...  # Node feature matrix: [num_nodes, num_features]
# edge_index = ... # Edge indices: [2, num_edges]

src, dst = data.edge_index
score = (data.x[src] * data.x[dst]).sum(dim=-1)


In [19]:
score

tensor([264.1794, 263.3403, 263.5314,  ..., 118.7073, 118.9811, 104.0850])

In [18]:
# Test the model
model.eval()
with torch.no_grad():
    output = model(graph_data.x, graph_data.edge_index)
    pred = output.argmax(dim=1)
    correct = pred.eq(graph_data.y).sum().item()
    accuracy = correct / graph_data.y.size(0)
    print('Accuracy: {:.4f}'.format(accuracy))

torch.Size([6634, 64])
Accuracy: 0.9260
